**Prepare Environment & Download Models**

In [1]:
# @title

!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 numpy opencv-python==4.9.0.80 pillow tqdm

!pip install -r https://raw.githubusercontent.com/Isi-dev/animate-x/main/requirements.txt

!pip install onnxruntime-gpu

!pip install pynvml
!pip install kornia


!git clone https://github.com/Isi-dev/animate-x.git
%cd animate-x


!mkdir -p checkpoints

# Download DWPose models
!wget -O checkpoints/dw-ll_ucoco_384.onnx "https://huggingface.co/Isi99999/UniAnimate_and_Animate-X_Models/resolve/main/dw-ll_ucoco_384.onnx"
!wget -O checkpoints/yolox_l.onnx "https://huggingface.co/Isi99999/UniAnimate_and_Animate-X_Models/resolve/main/yolox_l.onnx"

# Download Animate-X, Embedder & AutoEncoder models
!wget -O checkpoints/animate-x_ckpt.pth "https://huggingface.co/Isi99999/UniAnimate_and_Animate-X_Models/resolve/main/animate-x_ckpt.pth"
!wget -O checkpoints/open_clip_pytorch_model.bin "https://huggingface.co/Isi99999/UniAnimate_and_Animate-X_Models/resolve/main/open_clip_pytorch_model.bin"
!wget -O checkpoints/v2-1_512-ema-pruned.ckpt "https://huggingface.co/Isi99999/UniAnimate_and_Animate-X_Models/resolve/main/v2-1_512-ema-pruned.ckpt"

print("✅ All models are ready!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196

**Load Inputs & Generate DWPose Data**

In [2]:
# @title
import os
import glob
import cv2
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, Video, HTML
import PIL.Image
from base64 import b64encode
from IPython.display import clear_output


os.makedirs("data/videos", exist_ok=True)
os.makedirs("data/images", exist_ok=True)
os.makedirs("data/saved_pkl", exist_ok=True)
os.makedirs("data/saved_pose", exist_ok=True)
os.makedirs("data/saved_frames", exist_ok=True)


upload_image = widgets.FileUpload(accept="image/*", multiple=False)
upload_video = widgets.FileUpload(accept="video/*", multiple=False)
run_button = widgets.Button(description="Extract DWPose", button_style="success")
output_display = widgets.Output()

print("Upload Reference Image:")
display(upload_image)
print("\nUpload Driving Video:")
display(upload_video)



def save_uploaded_files(upload_widget, save_dir):
    if not upload_widget.value:
        return None
    for filename, file_info in upload_widget.value.items():
        file_path = os.path.join(save_dir, filename)
        with open(file_path, 'wb') as f:
            f.write(file_info['content'])
        return file_path


def on_run_button_click(b):
    clear_output(wait=True)

    with output_display:
        output_display.clear_output()


        image_path = save_uploaded_files(upload_image, "data/images")
        video_path = save_uploaded_files(upload_video, "data/videos")

        if not video_path:
            print("❌ No driving video uploaded!")
            return

        print(f"✅ Reference Image Saved: {image_path}")
        print(f"✅ Driving Video Saved: {video_path}")
        print("🔄 Running DWPose extraction...")

        def check_and_install(package):
          try:
              __import__(package)
          except ImportError:
              print(f"🔍 {package} not found. Installing...")
              import subprocess
              import sys
              subprocess.check_call([sys.executable, "-m", "pip", "install", package])
              print(f"✅ {package} installed successfully.")


        check_and_install("onnxruntime")


        !python process_data.py --source_video_paths data/videos --saved_pose_dir data/saved_pkl --saved_pose data/saved_pose --saved_frame_dir data/saved_frames

        print("✅ DWPose data extraction complete!")

        # Process Extracted Frames
        # frame_paths = sorted(glob.glob("data/saved_pose/*/*.jpg"))  # Include subdirectories

        # if not frame_paths:
        #     print("❌ No extracted frames found. Ensure process_data.py ran successfully.")
        # else:
        #
        #
        #     first_frame = cv2.imread(frame_paths[0])
        #     if first_frame is None:
        #         print("❌ First frame is blank. Video cannot be generated.")
        #     else:
        #         print("First frame loaded correctly.")

        #
        #     video_output_path = "dwpose_output.mp4"
        #     heightV, widthV = first_frame.shape[:2]
        #     print(f"width of video: {widthV}")
        #     print(f"height of video: {heightV}")
        #     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        #     video = cv2.VideoWriter(video_output_path, fourcc, 30, (widthV, heightV))

        #     for frame_path in frame_paths:
        #         img = cv2.imread(frame_path, cv2.IMREAD_UNCHANGED)
        #         if img is None:
        #             print(f"❌ Skipping blank frame: {frame_path}")
        #             continue  # Skip blank frames

        #         if img.shape[2] == 4:  # Convert RGBA to BGR if needed
        #             print("Converting frames to BGR...")
        #             img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        #         video.write(img)

        #     video.release()
        #     print("✅ Video generation complete.")

        #
        #     print("✅ Extracted DWPose video:")
        #     display(Video(video_output_path, embed=True))


run_button.on_click(on_run_button_click)

display(run_button, output_display)


Upload Reference Image:


FileUpload(value={}, accept='image/*', description='Upload')


Upload Driving Video:


FileUpload(value={}, accept='video/*', description='Upload')

Button(button_style='success', description='Extract DWPose', style=ButtonStyle())

Output()

**Set Inference Parameters & Run**

In [3]:
# @title
import os
import yaml
import glob
import IPython.display as display
import ipywidgets as widgets
from IPython.display import clear_output


def run_inference(_):

    clear_output(wait=True)

    import glob

    max_frames = max_frames_input.value
    resolution = [int(x) for x in resolution_input.value.split(",")]
    round_val = round_input.value
    ddim_timesteps = ddim_timesteps_input.value
    seed = seed_input.value
    save_fps = save_fps_input.value
    n = frame_skip_input.value


    image_files = sorted(glob.glob("data/images/*"))
    saved_pose_folders = sorted(glob.glob("data/saved_pose/*"))
    saved_frame_folders = sorted(glob.glob("data/saved_frames/*"))
    saved_pkl_files = sorted(glob.glob("data/saved_pkl/*.pkl"))


    test_list_path = []
    for img_file, pose_folder, frame_folder, pkl_file in zip(image_files, saved_pose_folders, saved_frame_folders, saved_pkl_files):
        test_list_path.append([n, img_file, pose_folder, frame_folder, pkl_file, seed])


    config = {
        "max_frames": max_frames,
        "resolution": resolution,
        "round": round_val,
        "ddim_timesteps": ddim_timesteps,
        "seed": seed,
        "save_fps": save_fps,
        "test_list_path": test_list_path,
        "log_dir": "results",
        "test_model": "checkpoints/animate-x_ckpt.pth",
        "partial_keys": [["image", "local_image", "dwpose", "pose_embeddings"]],
        "TASK_TYPE": "inference_animate_x_entrance",
        "use_fp16": True,
        "guide_scale": 2.5,
        "vit_resolution": [224, 224],
        "batch_size": 1,
        "latent_random_ref": True,
        "chunk_size": 2,
        "decoder_bs": 2,
        "scale": 8,
        "use_fps_condition": False,
        "embedder": {
            "type": "FrozenOpenCLIPTextVisualEmbedder",
            "layer": "penultimate",
            "pretrained": "checkpoints/open_clip_pytorch_model.bin"
        },
        "auto_encoder": {
            "type": "AutoencoderKL",
            "ddconfig": {
                "double_z": True,
                "z_channels": 4,
                "resolution": 256,
                "in_channels": 3,
                "out_ch": 3,
                "ch": 128,
                "ch_mult": [1, 2, 4, 4],
                "num_res_blocks": 2,
                "attn_resolutions": [],
                "dropout": 0.0,
                "video_kernel_size": [3, 1, 1]
            },
            "embed_dim": 4,
            "pretrained": "checkpoints/v2-1_512-ema-pruned.ckpt"
        },
        "UNet": {
            "type": "UNetSD_Animate_X",
            "config": None,
            "in_dim": 4,
            "num": 0,
            "no_hand": True,
            "dim": 320,
            "y_dim": 1024,
            "context_dim": 1024,
            "out_dim": 4,
            "dim_mult": [1, 2, 4, 4],
            "num_heads": 8,
            "head_dim": 64,
            "num_res_blocks": 2,
            "dropout": 0.1,
            "temporal_attention": True,
            "num_tokens": 4,
            "temporal_attn_times": 1,
            "use_checkpoint": True,
            "use_fps_condition": False,
            "use_sim_mask": False,
            "seq_len": max_frames + 1
        },
        "video_compositions": ["image", "local_image", "dwpose", "randomref", "randomref_pose", "pose_embedding"],
        "Diffusion": {
            "type": "DiffusionDDIM",
            "schedule": "linear_sd",
            "schedule_param": {
                "num_timesteps": 1000,
                "init_beta": 0.00085,
                "last_beta": 0.0120,
                "zero_terminal_snr": True
            },
            "mean_type": "v",
            "loss_type": "mse",
            "var_type": "fixed_small",
            "rescale_timesteps": False,
            "noise_strength": 0.1
        },
        "use_DiffusionDPM": False,
        "CPU_CLIP_VAE": True
    }


    config_path = "configs/Animate_X_infer.yaml"
    with open(config_path, "w") as file:
        yaml.dump(config, file, default_flow_style=False, sort_keys=False)

    print(f"✅ Configuration file updated: {config_path}")


    !python inference.py --cfg configs/Animate_X_infer.yaml

    import os
    import glob
    from IPython.display import display as displayVid, Video as outVid


    video_folder = "results/Animate_X_infer/"


    video_files = glob.glob(os.path.join(video_folder, "*.mp4"))

    if video_files:
        latest_video = max(video_files, key=os.path.getctime)
        print(f"Displaying video: {latest_video}")
        displayVid(outVid(latest_video, embed=True))
    else:
        print("❌ No video found in results/Animate_X_infer/")


max_frames_input = widgets.IntText(value=32, description="Max Frames:")
resolution_input = widgets.Text(value="512,768", description="Resolution:")
round_input = widgets.IntText(value=1, description="Round:")
ddim_timesteps_input = widgets.IntText(value=30, description="DDIM Steps:")
seed_input = widgets.IntText(value=13, description="Seed:")
save_fps_input = widgets.FloatText(value=8.0, description="FPS:", step=0.5)
frame_skip_input = widgets.IntText(value=1, description="Frame Skip (n):")


run_button = widgets.Button(description="Run Inference", button_style="danger")
run_button.on_click(run_inference)


widgets_box = widgets.VBox([max_frames_input, resolution_input, round_input, ddim_timesteps_input, seed_input, save_fps_input, frame_skip_input, run_button])
display.display(widgets_box)
